# Paper 28: Dense Passage Retrieval for Open-Domain Question Answering
## Vladimir Karpukhin, Barlas Oğuz, Sewon Min, et al., Meta AI (2020)

### Dense Passage Retrieval (DPR)

Learn dense embeddings for questions and passages. Retrieve via similarity in embedding space. Beats BM25!

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from collections import Counter
import re

np.random.seed(42)

## Dual Encoder Architecture

```
Question → Encoder_Q → q (dense vector)
Passage  → Encoder_P → p (dense vector)

Similarity: sim(q, p) = q · p  (dot product)
```

In [ ]:
class SimpleTextEncoder:
    """Simplified text encoder (in practice: use BERT)"""
    def __init__(self, vocab_size, embedding_dim, hidden_dim):
        self.vocab_size = vocab_size
        self.embedding_dim = embedding_dim
        self.hidden_dim = hidden_dim
        
        # Embeddings
        self.embeddings = np.random.randn(vocab_size, embedding_dim) * 0.01
        
        # Simple RNN weights
        self.W_xh = np.random.randn(hidden_dim, embedding_dim) * 0.01
        self.W_hh = np.random.randn(hidden_dim, hidden_dim) * 0.01
        self.b_h = np.zeros((hidden_dim, 1))
        
        # Output projection
        self.W_out = np.random.randn(hidden_dim, hidden_dim) * 0.01
    
    def encode(self, token_ids):
        """
        Encode sequence of token IDs to dense vector
        Returns: dense embedding (hidden_dim,)
        """
        h = np.zeros((self.hidden_dim, 1))
        
        # Process tokens
        for token_id in token_ids:
            # Lookup embedding
            x = self.embeddings[token_id].reshape(-1, 1)
            
            # RNN step
            h = np.tanh(np.dot(self.W_xh, x) + np.dot(self.W_hh, h) + self.b_h)
        
        # Final representation (CLS-like)
        output = np.dot(self.W_out, h).flatten()
        
        # L2 normalize for cosine similarity
        output = output / (np.linalg.norm(output) + 1e-8)
        
        return output

# Create encoders
vocab_size = 1000
embedding_dim = 64
hidden_dim = 128

question_encoder = SimpleTextEncoder(vocab_size, embedding_dim, hidden_dim)
passage_encoder = SimpleTextEncoder(vocab_size, embedding_dim, hidden_dim)

# Test
test_tokens = [10, 25, 37, 42]
q_emb = question_encoder.encode(test_tokens)
p_emb = passage_encoder.encode(test_tokens)

print(f"Question embedding shape: {q_emb.shape}")
print(f"Passage embedding shape: {p_emb.shape}")
print(f"Similarity (dot product): {np.dot(q_emb, p_emb):.4f}")

## Synthetic QA Dataset

In [ ]:
class SimpleTokenizer:
    """Simple word tokenizer"""
    def __init__(self):
        self.word_to_id = {}
        self.id_to_word = {}
        self.next_id = 0
    
    def tokenize(self, text):
        """Convert text to token IDs"""
        words = text.lower().split()
        token_ids = []
        
        for word in words:
            if word not in self.word_to_id:
                self.word_to_id[word] = self.next_id
                self.id_to_word[self.next_id] = word
                self.next_id += 1
            token_ids.append(self.word_to_id[word])
        
        return token_ids

# Create synthetic dataset
passages = [
    "The Eiffel Tower is a wrought-iron lattice tower in Paris, France.",
    "The Great Wall of China is a series of fortifications in northern China.",
    "The Statue of Liberty is a colossal neoclassical sculpture in New York.",
    "The Colosseum is an oval amphitheatre in the centre of Rome, Italy.",
    "The Taj Mahal is an ivory-white marble mausoleum in Agra, India.",
    "Mount Everest is Earth's highest mountain above sea level.",
    "The Amazon River is the largest river by discharge volume of water.",
    "The Sahara is a desert on the African continent.",
]

questions = [
    ("What is the Eiffel Tower?", 0),  # (question, relevant_passage_idx)
    ("Where is the Great Wall located?", 1),
    ("What is the tallest mountain?", 5),
    ("Where is the Statue of Liberty?", 2),
    ("What is the largest river?", 6),
]

# Tokenize
tokenizer = SimpleTokenizer()

passage_tokens = [tokenizer.tokenize(p) for p in passages]
question_tokens = [(tokenizer.tokenize(q), idx) for q, idx in questions]

print("Sample passage:")
print(f"Text: {passages[0]}")
print(f"Tokens: {passage_tokens[0][:10]}...")
print(f"\nVocabulary size: {tokenizer.next_id}")

## Encode Corpus and Questions

In [ ]:
# Re-initialize encoders with correct vocab size
vocab_size = tokenizer.next_id
question_encoder = SimpleTextEncoder(vocab_size, embedding_dim=32, hidden_dim=64)
passage_encoder = SimpleTextEncoder(vocab_size, embedding_dim=32, hidden_dim=64)

# Encode all passages
passage_embeddings = []
for tokens in passage_tokens:
    emb = passage_encoder.encode(tokens)
    passage_embeddings.append(emb)
passage_embeddings = np.array(passage_embeddings)

# Encode questions
question_embeddings = []
for tokens, _ in question_tokens:
    emb = question_encoder.encode(tokens)
    question_embeddings.append(emb)
question_embeddings = np.array(question_embeddings)

print(f"Passage embeddings: {passage_embeddings.shape}")
print(f"Question embeddings: {question_embeddings.shape}")

## Dense Retrieval via Maximum Inner Product Search (MIPS)

In [ ]:
def retrieve_top_k(query_embedding, passage_embeddings, k=3):
    """
    Retrieve top-k passages for query
    Uses dot product similarity (MIPS)
    """
    # Compute similarities
    similarities = np.dot(passage_embeddings, query_embedding)
    
    # Get top-k indices
    top_k_indices = np.argsort(similarities)[::-1][:k]
    top_k_scores = similarities[top_k_indices]
    
    return top_k_indices, top_k_scores

# Test retrieval
print("\nDense Retrieval Results:\n" + "="*80)
for i, (q_tokens, correct_idx) in enumerate(question_tokens):
    question_text = questions[i][0]
    q_emb = question_embeddings[i]
    
    # Retrieve
    top_indices, top_scores = retrieve_top_k(q_emb, passage_embeddings, k=3)
    
    print(f"\nQ: {question_text}")
    print(f"Correct passage: #{correct_idx}")
    print(f"\nRetrieved (top-3):")
    for rank, (idx, score) in enumerate(zip(top_indices, top_scores), 1):
        is_correct = "✓" if idx == correct_idx else "✗"
        print(f"  {rank}. [{is_correct}] (score={score:.3f}) {passages[idx][:60]}...")

print("\n" + "="*80)
print("(Encoders are untrained, so results are random)")

## Training with In-Batch Negatives

In [ ]:
def softmax(x):
    exp_x = np.exp(x - np.max(x))  # Numerical stability
    return exp_x / np.sum(exp_x)

def contrastive_loss(query_emb, positive_emb, negative_embs):
    """
    Contrastive loss (InfoNCE)
    
    L = -log( exp(q·p+) / (exp(q·p+) + Σ exp(q·p-)) )
    """
    # Positive score
    pos_score = np.dot(query_emb, positive_emb)
    
    # Negative scores
    neg_scores = [np.dot(query_emb, neg_emb) for neg_emb in negative_embs]
    
    # All scores
    all_scores = np.array([pos_score] + neg_scores)
    
    # Softmax
    probs = softmax(all_scores)
    
    # Negative log likelihood (positive should be first)
    loss = -np.log(probs[0] + 1e-8)
    
    return loss

# Simulate training batch
batch_size = 3
batch_questions = question_embeddings[:batch_size]
batch_passages = passage_embeddings[:batch_size]

# In-batch negatives: for each question, other passages in batch are negatives
total_loss = 0
print("\nIn-Batch Negative Training:\n" + "="*80)
for i in range(batch_size):
    q_emb = batch_questions[i]
    pos_emb = batch_passages[i]  # Correct passage
    
    # Negatives: all other passages in batch
    neg_embs = [batch_passages[j] for j in range(batch_size) if j != i]
    
    loss = contrastive_loss(q_emb, pos_emb, neg_embs)
    total_loss += loss
    
    print(f"Question {i}: loss = {loss:.4f}")

avg_loss = total_loss / batch_size
print(f"\nAverage batch loss: {avg_loss:.4f}")
print("\nIn-batch negatives: efficient hard negative mining!")

## Visualize Embedding Space

In [ ]:
# Simple 2D projection (PCA-like)
def project_2d(embeddings):
    """Project high-dim embeddings to 2D (simplified PCA)"""
    # Mean center
    mean = np.mean(embeddings, axis=0)
    centered = embeddings - mean
    
    # Take first 2 principal components (simplified)
    U, S, Vt = np.linalg.svd(centered, full_matrices=False)
    projected = U[:, :2] * S[:2]
    
    return projected

# Project to 2D
all_embeddings = np.vstack([passage_embeddings, question_embeddings])
projected = project_2d(all_embeddings)

passage_2d = projected[:len(passage_embeddings)]
question_2d = projected[len(passage_embeddings):]

# Visualize
plt.figure(figsize=(12, 10))

# Plot passages
plt.scatter(passage_2d[:, 0], passage_2d[:, 1], s=200, c='lightblue', 
           edgecolors='black', linewidths=2, marker='s', label='Passages', zorder=2)

# Annotate passages
for i, (x, y) in enumerate(passage_2d):
    plt.text(x, y-0.15, f'P{i}', ha='center', fontsize=10, fontweight='bold')

# Plot questions
plt.scatter(question_2d[:, 0], question_2d[:, 1], s=200, c='lightcoral', 
           edgecolors='black', linewidths=2, marker='o', label='Questions', zorder=3)

# Annotate questions
for i, (x, y) in enumerate(question_2d):
    plt.text(x, y+0.15, f'Q{i}', ha='center', fontsize=10, fontweight='bold')

# Draw connections (question to correct passage)
for i, (q_tokens, correct_idx) in enumerate(question_tokens):
    q_pos = question_2d[i]
    p_pos = passage_2d[correct_idx]
    plt.plot([q_pos[0], p_pos[0]], [q_pos[1], p_pos[1]], 
            'g--', alpha=0.5, linewidth=2, label='Correct' if i == 0 else '')

plt.xlabel('Dimension 1', fontsize=12)
plt.ylabel('Dimension 2', fontsize=12)
plt.title('Dense Retrieval Embedding Space (2D Projection)', fontsize=14, fontweight='bold')
plt.legend(fontsize=11)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

print("\nIdeal: Questions close to their relevant passages!")

## Compare with BM25 (Sparse Retrieval)

In [ ]:
class SimpleBM25:
    """Simplified BM25 scoring"""
    def __init__(self, passages, k1=1.5, b=0.75):
        self.passages = passages
        self.k1 = k1
        self.b = b
        
        # Compute document frequencies
        self.doc_freqs = {}
        self.avg_doc_len = 0
        
        all_words = []
        for passage in passages:
            words = set(passage.lower().split())
            all_words.extend(passage.lower().split())
            for word in words:
                self.doc_freqs[word] = self.doc_freqs.get(word, 0) + 1
        
        self.avg_doc_len = len(all_words) / len(passages)
        self.N = len(passages)
    
    def score(self, query, passage_idx):
        """BM25 score for query and passage"""
        query_words = query.lower().split()
        passage = self.passages[passage_idx]
        passage_words = passage.lower().split()
        passage_len = len(passage_words)
        
        # Count term frequencies
        tf = Counter(passage_words)
        
        score = 0
        for word in query_words:
            if word not in tf:
                continue
            
            # IDF
            df = self.doc_freqs.get(word, 0)
            idf = np.log((self.N - df + 0.5) / (df + 0.5) + 1)
            
            # TF component
            freq = tf[word]
            norm = 1 - self.b + self.b * (passage_len / self.avg_doc_len)
            tf_component = (freq * (self.k1 + 1)) / (freq + self.k1 * norm)
            
            score += idf * tf_component
        
        return score
    
    def retrieve(self, query, k=3):
        """Retrieve top-k passages for query"""
        scores = [self.score(query, i) for i in range(len(self.passages))]
        top_k_indices = np.argsort(scores)[::-1][:k]
        top_k_scores = [scores[i] for i in top_k_indices]
        return top_k_indices, top_k_scores

# Create BM25 retriever
bm25 = SimpleBM25(passages)

# Compare BM25 vs Dense
print("\nBM25 vs Dense Retrieval Comparison:\n" + "="*80)
for i, (question_text, correct_idx) in enumerate(questions):
    print(f"\nQ: {question_text}")
    print(f"Correct: #{correct_idx}")
    
    # BM25
    bm25_indices, bm25_scores = bm25.retrieve(question_text, k=3)
    print(f"\nBM25 Top-3:")
    for rank, (idx, score) in enumerate(zip(bm25_indices, bm25_scores), 1):
        is_correct = "✓" if idx == correct_idx else "✗"
        print(f"  {rank}. [{is_correct}] (score={score:.3f}) #{idx}")
    
    # Dense
    q_emb = question_embeddings[i]
    dense_indices, dense_scores = retrieve_top_k(q_emb, passage_embeddings, k=3)
    print(f"\nDense Top-3:")
    for rank, (idx, score) in enumerate(zip(dense_indices, dense_scores), 1):
        is_correct = "✓" if idx == correct_idx else "✗"
        print(f"  {rank}. [{is_correct}] (score={score:.3f}) #{idx}")

print("\n" + "="*80)
print("BM25: Lexical matching (sparse)")
print("Dense: Semantic matching (dense embeddings)")

## Retrieval Metrics

In [ ]:
def compute_metrics(predictions, correct_indices, k_values=[1, 3, 5]):
    """
    Compute retrieval metrics:
    - Recall@k: % of queries where correct passage is in top-k
    - MRR (Mean Reciprocal Rank): average 1/rank of correct passage
    """
    n_queries = len(predictions)
    
    recalls = {k: 0 for k in k_values}
    reciprocal_ranks = []
    
    for pred, correct_idx in zip(predictions, correct_indices):
        # Find rank of correct passage
        if correct_idx in pred:
            rank = list(pred).index(correct_idx) + 1
            reciprocal_ranks.append(1.0 / rank)
            
            # Update recall@k
            for k in k_values:
                if rank <= k:
                    recalls[k] += 1
        else:
            reciprocal_ranks.append(0.0)
    
    # Compute averages
    mrr = np.mean(reciprocal_ranks)
    recalls = {k: v / n_queries for k, v in recalls.items()}
    
    return recalls, mrr

# Evaluate both methods
bm25_predictions = []
dense_predictions = []
correct_indices = []

for i, (question_text, correct_idx) in enumerate(questions):
    # BM25
    bm25_top, _ = bm25.retrieve(question_text, k=5)
    bm25_predictions.append(bm25_top)
    
    # Dense
    q_emb = question_embeddings[i]
    dense_top, _ = retrieve_top_k(q_emb, passage_embeddings, k=5)
    dense_predictions.append(dense_top)
    
    correct_indices.append(correct_idx)

# Compute metrics
bm25_recalls, bm25_mrr = compute_metrics(bm25_predictions, correct_indices)
dense_recalls, dense_mrr = compute_metrics(dense_predictions, correct_indices)

# Display
print("\nRetrieval Metrics:\n" + "="*60)
print(f"{'Metric':<15} {'BM25':<15} {'Dense':<15}")
print("-" * 60)
for k in [1, 3, 5]:
    print(f"Recall@{k:<10} {bm25_recalls[k]:<15.2%} {dense_recalls[k]:<15.2%}")
print(f"MRR{'':<12} {bm25_mrr:<15.3f} {dense_mrr:<15.3f}")
print("="*60)
print("\n(Models are untrained - results are random)")

## Key Takeaways

### Dense Passage Retrieval (DPR) Architecture:

**Dual Encoder**:
```
Question: q → BERT_Q → E_Q(q) = q_emb
Passage:  p → BERT_P → E_P(p) = p_emb

Similarity: sim(q, p) = q_emb · p_emb
```

### Training Objective:

**Contrastive Loss (InfoNCE)**:
$$
L(q_i, p_i^+, p_i^{-1}, ..., p_i^{-n}) = -\log \frac{e^{\text{sim}(q_i, p_i^+)}}{e^{\text{sim}(q_i, p_i^+)} + \sum_j e^{\text{sim}(q_i, p_i^{-j})}}
$$

Where:
- $p_i^+$: Positive (relevant) passage
- $p_i^{-j}$: Negative (irrelevant) passages

### In-Batch Negatives:

Efficient negative mining:
```
Batch: [(q1, p1+), (q2, p2+), ..., (qB, pB+)]

For q1:
  Positive: p1+
  Negatives: p2+, p3+, ..., pB+ (from other examples)
```

**Benefits**:
- No extra passages needed
- Gradient flows through all examples
- Scales to large batch sizes

### Hard Negative Mining:

1. **BM25 negatives**: Top BM25 results that aren't relevant
2. **Random negatives**: Random passages from corpus
3. **In-batch negatives**: Other positives in batch

**Best**: Combine all three!

### Inference (Retrieval):

**Offline**:
1. Encode all passages: $P = \{E_P(p_1), ..., E_P(p_N)\}$
2. Build MIPS index (e.g., FAISS)

**Online** (at query time):
1. Encode query: $q_{emb} = E_Q(q)$
2. Search index: top-k by $\arg\max_p \, q_{emb} \cdot p_{emb}$

### DPR vs BM25:

| Aspect | BM25 | DPR |
|--------|------|-----|
| Matching | Lexical (exact words) | Semantic (meaning) |
| Training | None (heuristic) | Learned from data |
| Robustness | Sensitive to wording | Handles paraphrases |
| Speed | Fast (sparse) | Fast with MIPS index |
| Memory | Low | High (dense vectors) |

### Results (from paper):

**Natural Questions**:
- BM25: 59.1% Top-20 accuracy
- DPR: 78.4% Top-20 accuracy

**WebQuestions**:
- BM25: 55.0%
- DPR: 75.0%

**TREC**:
- BM25: 70.9%
- DPR: 79.4%

### Implementation Details:

1. **Encoders**: BERT-base (110M params)
2. **Embedding dim**: 768 (BERT hidden size)
3. **Batch size**: 128 (large for in-batch negatives)
4. **Hard negatives**: 1 BM25 + 1 random per positive
5. **Training**: ~40 epochs on 59k QA pairs

### Advantages:

- ✅ **Semantic matching**: Understands meaning, not just words
- ✅ **End-to-end**: Learned from question-passage pairs
- ✅ **Handles paraphrases**: "tallest mountain" = "highest peak"
- ✅ **Scalable**: MIPS with FAISS for billions of passages
- ✅ **Outperforms BM25**: +15-20% absolute accuracy

### Limitations:

- ❌ **Requires training data**: Need QA pairs
- ❌ **Memory**: Dense vectors for all passages
- ❌ **Index updates**: Re-encode when corpus changes
- ❌ **May miss exact matches**: BM25 better for rare entities

### Best Practices:

1. **Hybrid retrieval**: Combine BM25 + DPR
2. **Large batches**: More in-batch negatives
3. **Hard negatives**: Use BM25 top results
4. **Fine-tune**: Domain-specific data improves results
5. **FAISS**: Use for fast MIPS at scale

### Modern Extensions:

- **ColBERT**: Late interaction for better ranking
- **ANCE**: Approximate nearest neighbor negatives
- **RocketQA**: Cross-batch negatives
- **Contriever**: Unsupervised dense retrieval
- **Dense X Retrieval**: Multi-vector representations

### Applications:

- Open-domain QA (e.g., Google search)
- RAG (Retrieval-Augmented Generation)
- Document search
- Semantic search
- Knowledge base completion